In [1]:
# Import Comet and create experiment
from comet_ml import Experiment

# Comet variables
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import concatenate

API_KEY = "gKo0lOrCw6burIxXrjggq8OMY"
PROJECT = "MultiOutputNormed"
WORKSPACE = "PresetGen"
VSTPARAMETER = 25

experiment = Experiment(api_key=API_KEY, project_name=PROJECT, workspace=WORKSPACE, log_code=True)
experiment.set_code()
# Dependencies
import xml.etree.ElementTree as ET
import re
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib
import tensorflow as tf
import keras as keras
import keras.layers as layers
import kapre as kapre
import joblib as joblib

# matplotlib.use('agg')
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # no graka...
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers



COMET INFO: Experiment is live on comet.ml https://www.comet.ml/presetgen/multioutputnormed/c0e0c1d85d714853801e918d222af116



In [2]:
ACTUALEXNAME = "mfccs5Second_Env10_MultipleBatchMinMax" + datetime.now().strftime("%m%d%Y%H%M%S")
print(ACTUALEXNAME)
# Dependencies
SR = 22050


def getOneLinerMfcc(mfcc):
    new = np.mean(mfcc, axis=0)
    return new


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T, axis=0)
        SR = sample_rate


    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    return mfccsscaled


def getAudio(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        SR = sample_rate
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    return audio


def Conv1D(SR=SR, DT=1.0):
    i = layers.Input(shape=(1, int(SR * DT)), name='input')
    x = kapre.Melspectrogram(n_dft=512, n_hop=160,
                             padding='same', sr=SR, n_mels=128,
                             fmin=0.0, fmax=SR / 2, power_melgram=2.0,
                             return_decibel_melgram=True, trainable_fb=False,
                             trainable_kernel=False,
                             name='melbands')(i)
    x = kapre.Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Permute((2, 1, 3), name='permute')(x)
    x = kapre.TimeDistributed(layers.Conv1D(8, kernel_size=(4), activation='tanh'), name='td_conv_1d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_1')(x)
    x = kapre.TimeDistributed(layers.Conv1D(16, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_2')(x)
    x = kapre.TimeDistributed(layers.Conv1D(32, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_3')(x)
    x = kapre.TimeDistributed(layers.Conv1D(64, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name='max_pool_2d_4')(x)
    x = kapre.TimeDistributed(layers.Conv1D(128, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_4')(x)
    x = layers.GlobalMaxPooling2D(name='global_max_pooling_2d')(x)
    x = layers.Dropout(rate=0.1, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=keras.regularizers.l2(0.001), name='dense')(x)
    o = layers.Dense(1, name='dense_output')(x)

    model1 = kapre.Model(inputs=i, outputs=o, name='1d_convolution')

    return model1




mfccs5Second_Env10_MultipleBatchMinMax07152020145158


In [5]:
# Set the path to the full UrbanSound dataset
fulldatasetpath = "I://Synth1PresetTestFiles"

features_mfcc = []
features_env10 = []

testserializer = []

dirFiles = os.listdir("I://Synth1PresetTestFiles")
dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))
env10 = joblib.load("I://EnvelopeDiv10.dat")
# mfccss = []
# mfccss = np.load("I://zeroCrossFromPreset_frame64hop16.dat", allow_pickle=True)
mfccss1 = joblib.load("I://mfccFromPreset_128fft256hop64_1.dat")
mfccss2 = joblib.load("I://mfccFromPreset_128fft256hop64_2.dat")
mfccss = np.append(mfccss1, mfccss2)[1::2]
del mfccss1, mfccss2

value = 0

for file in dirFiles:
    if "xml" in file:
        sampleNumber = int(file.split(".")[0][4:])
        tempName = file.split(".")[0] + ".wav"
        fileName = os.path.join(os.path.abspath(fulldatasetpath), tempName)
        tree = ET.parse(os.path.join(os.path.abspath(fulldatasetpath), file))
        root = tree.getroot()
        classlabels = []
        for x in range(99):
            classlabels.append(int(root[x + 2].attrib["presetValue"]))
        #        classLabel = int(root[value + 2].attrib["presetValue"])
        data_mfcc = mfccss[sampleNumber]
        data_env10 = env10[sampleNumber, 1]
        features_mfcc.append([data_mfcc, classlabels])
        features_env10.append([data_env10, classlabels])
        value = value + 1
        # print("Appended " + fileName + " Class = " + str(classlabels) + " SampleNumber = " + str(sampleNumber))

del dirFiles
print("All Data Appended")
# np.array(mfccss).dump("mfccsFromPreset.dat")

All Data Appended


In [8]:

# Convert into a Panda dataframe
features_mffc_new = pd.DataFrame(features_mfcc, columns=['feature', 'class_label'])
features_env10_new = pd.DataFrame(features_env10, columns=['feature', 'class_label'])


# Convert features and corresponding classification labels into numpy arrays
featureArray_mfcc = np.array(features_mffc_new.feature.tolist())
featureArray_env10 = np.array(features_env10_new.feature.tolist())
valueArray = np.array(features_mffc_new.class_label.tolist())
# valueArray = tf.keras.utils.normalize(valueArray, axis=0, order=2)
# min_d = np.min(valueArray)
# max_d = np.max(valueArray)
# valueArray = (valueArray - min_d) / (max_d - min_d)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(valueArray)
valueArray = scaler.transform(valueArray)
# Encode the classification labels
le = LabelEncoder()
# yy = to_categorical(le.fit_transform(valueArray))

# split the dataset
from sklearn.model_selection import train_test_split

del features_mfcc, features_env10
del mfccss, tree, root
x_train, x_test, y_train, y_test = train_test_split(featureArray_mfcc, valueArray, test_size=0.2, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(featureArray_env10, valueArray, test_size=0.2, random_state=42)




MemoryError: Unable to allocate 10.5 GiB for an array with shape (12772, 128, 1723) and data type float32

In [11]:

del features_mfcc, features_env10
del mfccss, tree, root
#del featureArray_mfcc, featureArray_env10, valueArray

features_mffc = 1


NameError: name 'features_mfcc' is not defined

In [ ]:

num_labels = valueArray.shape
filter_size = 2

# Construct model
# model = Sequential([
#     Dense(16, activation=tf.nn.leaky_relu, input_shape=(40, 988,)),
#     Dense(32, activation=tf.nn.leaky_relu),
#     layers.GlobalMaxPool1D(),
#     Dense(64, activation=tf.nn.leaky_relu),
#     Dense(1)
# ])
# model = Sequential([
#     Dense(16, activation=tf.nn.leaky_relu, input_shape=(128, 1723,)),
#     Dense(32, activation=tf.nn.leaky_relu),
#     layers.GlobalMaxPool1D(),
#     Dense(64, activation=tf.nn.leaky_relu),
#     Dense(99)
# ])
# model = Sequential()
# model.add(Dense(16, input_shape=((128, 1723,), (11025,)), activation="relu"))
# # model.add(keras.layers.BatchNormalization())
# # model.add(layers.Activation("relu"))
# model.add(Dense(32, activation="relu"))
# model.add(layers.GlobalMaxPool1D())
# model.add(Dense(64, activation=tf.nn.leaky_relu))
# model.add(Dense(99))

# define two sets of inputs
inputA = Input(shape=(128, 1723,))
inputB = Input(shape=(11025,))
# the first branch operates on the first input
x = Dense(16, activation="relu")(inputA)
x = Dense(32, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
x = Dense(64, activation=tf.nn.leaky_relu)(x)
#x = Dense(99)(x)
x = Model(inputs=inputA, outputs=x)

# the second branch opreates on the second input
y = Dense(16, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(64, activation=tf.nn.leaky_relu)(y)
#y = Dense(99)(y)
y = Model(inputs=inputB, outputs=y)

# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(99)(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

print(model.summary())
optimizer = optimizers.RMSprop(0.001)

model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy', 'mae'])

num_epochs = 100
num_batch_size = 32

start = datetime.now()

# model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# reduce_lr_callback = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)

# my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=2),
#     tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
# ]
print("start Training....")
history = model.fit(
    [x_train, x_train2], y_train, batch_size=32,
    epochs=num_epochs, validation_split=0.2, verbose=0) # validation_data=(x_test, y_test)

duration = datetime.now() - start
model.save(ACTUALEXNAME)
print(model.summary())

print("Training completed in time: ", duration)

print(ACTUALEXNAME)

score = model.evaluate(x_train, y_train, verbose=0)
print('train accuracy: {}'.format(score))
# experiment.log_metric("train_acc", score)

score = model.evaluate(x_test, y_test, verbose=0)
print('test accuracy: {}'.format(score))
# experiment.log_metric("val_acc", score)

# print("First 10 Value Predictions")
# exampleMfccs = x_test[:10]
#
# exampleresult = model.predict(exampleMfccs)
#
# for val in exampleresult:
#     print(val)


